In [1]:
import os

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

from openai import OpenAI

# Nebius uses the same OpenAI() class, but with additional details
nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

llm_model = "meta-llama/Meta-Llama-3.1-405B-Instruct"

In [2]:
def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n") #Split the chunk of text retrieved from LLM into lines
    for line in lines:       #Loop all the lines
        current_line = ""
        words = line.split() #Split the lines into words separate by whitespace
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

In [3]:
def answer_with_llm(prompt: str,
                    system_prompt,
                    max_tokens=512,
                    client=nebius_client,
                    model=llm_model,
                    prettify=True,
                    temperature=0.7) -> str:

    messages = []

    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    messages.append(
        {
            "role": "user",
            "content": prompt
        }
    )

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )
    if prettify:
        return prettify_string(completion.choices[0].message.content)
    else:
        return completion.choices[0].message.content

In [14]:
def translate_in_stages(input: str, language: str) -> str:
    print("=== LITERAL TRANSLATION ===")
    literal_translation = answer_with_llm(
        f"""Translate the following children's rhyme into {language}, 
word for word, focusing on preserving the original meaning. Do not try to make it rhyme:\n\n{input}""",
        "You are a professional translator."
    )
    print("A: "+literal_translation)

    print("\n=== RHYMED TRANSLATION ===")
    rhymed_translation = answer_with_llm(
        f"""Take the following literal translation of a rhyme in {language} and rewrite it so it has rhyme and rhythm, 
even if you must lose some literal meaning. Return only the revised poem:\n\n{literal_translation}""",
        "You are a skilled poet."
    )
    print("B: "+rhymed_translation)

    print("\n=== FINAL EDITED VERSION ===")
    final_version = answer_with_llm(
        f"""Here is the original English rhyme and a translated version 
in {language}:\n\nOriginal:\n{input}\n\nTranslation:\n{rhymed_translation}\n\nEdit the translation for clarity, 
beauty, rhyme, and flow, while keeping as much of the original meaning as possible. Return only the final poem.""",
        "You are an experienced editor of poetry."
    )
    print("C: "+final_version)
    return final_version


In [17]:
language = "French"
poem = """Humpty Dumpty sat on a wall,
Humpty Dumpty had a great fall.
All the king's horses and all the king's men
Couldn't put Humpty together again."""

result = translate_in_stages(poem, language)
print("\n=== RESULT ===")
print("D: "+result)

=== LITERAL TRANSLATION ===
A: Here's a word-for-word translation of the children's rhyme into French,
preserving the original meaning:

Humpty Dumpty était assis sur un mur,
Humpty Dumpty a eu une grande chute.
Tous les chevaux du roi et tous les hommes du roi
N'ont pas pu remettre Humpty ensemble à nouveau.

Note that I've kept the name 'Humpty Dumpty' as it is since it's a proper noun
and a well-known character, even in French-speaking countries.

=== RHYMED TRANSLATION ===
B: Humpty Dumpty sat on a wall so high,
He tumbled down with a sorrowful sigh.
The king's horses and men all came by,
But none could mend poor Humpty, and he wondered why.

=== FINAL EDITED VERSION ===
C: Humpty Dumpty s'assit sur un mur si haut,
Il tomba bas avec un triste cri sauvage.
Les chevaux et les hommes du roi vinrent tous,
Mais aucun ne put réparer l'infortuné, pauvre Humpty, jamais.

=== RESULT ===
D: Humpty Dumpty s'assit sur un mur si haut,
Il tomba bas avec un triste cri sauvage.
Les chevaux et les 